# Atlas Plot

In [ ]:
import holoviews as hv
hv.extension('bokeh')
import numpy as np
import geopandas as gpd
from scipy.spatial import ConvexHull, convex_hull_plot_2d
from bokeh.io import output_file, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.io import export_svg, export_png
from holoviews.operation.timeseries import rolling, rolling_outlier_std
from bokeh.io import output_file, show
from bokeh.plotting import figure
from sklearn.preprocessing import scale, normalize, MinMaxScaler
from utils.utils import chunk_creator, convex_hull
from scipy.stats import norm, kurtosis
from sklearn import preprocessing
from scipy import signal
from scipy.stats import skew, triang

In [ ]:
transect_unified = gpd.read_file('./Atlas_Dec2022/processing/Atlas_transects_final.gpkg', driver='GPKG')
faults_unified = gpd.read_file('./Atlas_Dec2022/processing/Atlas_faults_final.gpkg', driver='GPKG')

## Plotting displacement along Length

## Define data scalers


In [ ]:
x_scaler = MinMaxScaler(feature_range=(-1,1))
y_scaler = MinMaxScaler(feature_range=(-1,1))

In [ ]:
transect_unified['Fault ID'] = transect_unified['Fault ID'].astype('int')

In [ ]:
transect_unified_sorted = transect_unified.copy()
transect_unified_sorted.sort_values('Fault ID', inplace=True)
#transect_unified_sorted=transect_unified_sorted.reindex(transect_unified['Fault ID'].unique())
#transect_unified_sorted.drop('index',axis=1,inplace=True)

## Plot generation
### For each fault will be generated a combined plot for:
* MaxDisplacement/length
* ConvexHull results
* Rolling window

In [ ]:
profiles = []
cvs = []
import math
for jj in transect_unified_sorted['Fault ID'].unique():
    group_gdf = transect_unified_sorted[transect_unified_sorted['Fault ID']==jj]
    prof = sorted(list(zip(np.arange(0,int(group_gdf['Fault L(m)'].max()),len(group_gdf)),group_gdf['TrDsp (m)'])))
    smoothed = rolling(hv.Curve(prof), rolling_window=5).opts(color='red',labelled=['x'], alpha=0.5)

    xxx = np.array([xx[0] for xx in prof])#.flatten()
    yyy = np.array([yy[1] for yy in prof])#.flatten()
    xxx_norm = x_scaler.fit_transform(np.array(xxx).reshape(-1,1)).flatten()
    yyy_norm = y_scaler.fit_transform(np.array(yyy).reshape(-1,1)).flatten()
    points=list(zip(xxx_norm, yyy_norm))
    points=np.array([[i[0],i[1]] for i in points])
    
    ##convex hull
    #hull = ConvexHull(points)#, qhull_options=f'QG20')
    lower_hull, upper_hull = convex_hull(points=list(zip(xxx_norm, yyy_norm)))
    
    ### lower hull   
    #lhxs = np.array([lower_hull[i][0] for i in range(len(lower_hull))])
    #cxs +=[xs[0]]    
    #lhys = np.array([lower_hull[i][1] for i in range(len(lower_hull))])
    # cys +=[ys[0]]
    
    combo_hull = lower_hull + upper_hull
    chxs = np.array([combo_hull[i][0] for i in range(len(combo_hull))])
    chxs = np.insert(chxs, 0, 1)
    chxs = np.append(chxs, xxx_norm[0])
    chxs = np.append(chxs, -1)    
    chys = np.array([combo_hull[i][1] for i in range(len(combo_hull))])
    #stdev=np.std(chys)
    #mean=np.mean(chys)
    chys = np.insert(chys, 0, -1)
    chys = np.append(chys, yyy_norm[0])
    chys = np.append(chys, -1)
    
    #### upper hull
    uhxs = np.array([upper_hull[i][0] for i in range(len(upper_hull))])
    uhxs = np.insert(uhxs, 0, 1)
    uhxs = np.append(uhxs, xxx_norm[0])
    uhxs = np.append(uhxs, -1)    
    uhys = np.array([upper_hull[i][1] for i in range(len(upper_hull))])
    stdev=np.std(uhys)
    mean=np.mean(uhys)
    CV = round(mean/stdev,3)
    uhys = np.insert(uhys, 0, -1)
    uhys = np.append(uhys, yyy_norm[0])
    uhys = np.append(uhys, -1)
    ### mean and st.dev for upper hull
    #stdev=np.std(uhys)
    #mean=np.mean(uhys)
    ### plotting upper hull
    chplot = figure(x_axis_label = "Fault length (m)", y_axis_label= "Displacement (m)", width=500, height=250) 
    chplot.title.text=f'Convex Hull - Fault ID: {jj}, CV: {CV}'    
    chplot.border_fill_color = None
    chplot.output_backend = "svg"    
    chplot.xaxis.axis_label_text_font_size = "15pt"
    chplot.xaxis.major_label_text_font_size = "15pt"
    chplot.xaxis.axis_label_text_font = "times"
    chplot.xaxis.axis_label_text_color = "black"
    chplot.yaxis.axis_label_text_font_size = "15pt"
    chplot.yaxis.major_label_text_font_size = "15pt"
    chplot.yaxis.axis_label_text_font = "times"
    chplot.yaxis.axis_label_text_color = "black"
    chplot.yaxis.major_tick_line_width = 2
    chplot.yaxis.major_tick_out = 10
    chplot.yaxis.minor_tick_line_width = 2
    chplot.xaxis.major_tick_line_width = 2
    chplot.xaxis.major_tick_out = 10
    chplot.xaxis.minor_tick_line_width = 2
    chplot.xaxis.axis_line_width=2
    chplot.yaxis.axis_line_width=2
    chplot.xaxis.major_label_overrides = {-1: '-L/2', 0: '0', 1: 'L/2'}
    chplot.yaxis.major_label_overrides = {-1: '-D/2', 0: '0', 1: 'D/2'}
    chplot.line((xxx_norm),(yyy_norm),color='blue',width=1)
    #chplot.line((lhxs),(lhys),color='red',width=1.5)
    chplot.line((uhxs),(uhys),color='green',width=2)
    chplot.scatter(uhxs, uhys, color='orange')
    #######################
    
    ######## triangular fit
    triang_fit = triang.fit(yyy_norm)
    yyy_t = triang.pdf(xxx_norm, *triang_fit)
    trplot = figure(x_axis_label = "Fault length (m)", y_axis_label= "Displacement (m)", width=500, height=250) 
    trplot.title.text=f'Triangle fit - Fault ID: {jj}'  
    trplot.border_fill_color = None
    trplot.output_backend = "svg"    
    trplot.xaxis.axis_label_text_font_size = "15pt"
    trplot.xaxis.major_label_text_font_size = "15pt"
    trplot.xaxis.axis_label_text_font = "times"
    trplot.xaxis.axis_label_text_color = "black"
    trplot.yaxis.axis_label_text_font_size = "15pt"
    trplot.yaxis.major_label_text_font_size = "15pt"
    trplot.yaxis.axis_label_text_font = "times"
    trplot.yaxis.axis_label_text_color = "black"
    trplot.yaxis.major_tick_line_width = 2
    trplot.yaxis.major_tick_out = 10
    trplot.yaxis.minor_tick_line_width = 2
    trplot.xaxis.major_tick_line_width = 2
    trplot.xaxis.major_tick_out = 10
    trplot.xaxis.minor_tick_line_width = 2
    trplot.xaxis.axis_line_width=2
    trplot.yaxis.axis_line_width=2
    trplot.line(xxx_norm, yyy_norm, color='blue')
    trplot.line(xxx_norm, yyy_t, color='green', width=2)
    
    ############# ellipse fit
    from skimage.measure import EllipseModel
    from matplotlib.patches import Ellipse
    from ellipse import LsqEllipse
    #ell = EllipseModel()
    #ell.estimate(points)
    #xc, yc, a, b, theta = ell.params
    #print("center = ",  (xc, yc))
    #print("angle of rotation = ",  theta)
    #print("axes = ", (a,b))
    reg = LsqEllipse().fit(np.array(list(zip(xxx_norm,yyy_norm))))
    center, width, height, phi = reg.as_parameters()
    elplot = figure(x_axis_label = "Fault length (m)", y_axis_label= "Displacement (m)", width=500, height=250) 
    elplot.ellipse(x=center[0], y=center[1], width=2*width, height=2*height, angle=phi, fill_color="yellow", fill_alpha = 0, line_width=2, line_color='green') # * 180 / np.pi + 90
    
    elplot.title.text=f'Ellipse Fit - Fault ID: {jj}'  
    elplot.border_fill_color = None
    elplot.output_backend = "svg"    
    elplot.xaxis.axis_label_text_font_size = "15pt"
    elplot.xaxis.major_label_text_font_size = "15pt"
    elplot.xaxis.axis_label_text_font = "times"
    elplot.xaxis.axis_label_text_color = "black"
    elplot.yaxis.axis_label_text_font_size = "15pt"
    elplot.yaxis.major_label_text_font_size = "15pt"
    elplot.yaxis.axis_label_text_font = "times"
    elplot.yaxis.axis_label_text_color = "black"
    elplot.yaxis.major_tick_line_width = 2
    elplot.yaxis.major_tick_out = 10
    elplot.yaxis.minor_tick_line_width = 2
    elplot.xaxis.major_tick_line_width = 2
    elplot.xaxis.major_tick_out = 10
    elplot.xaxis.minor_tick_line_width = 2
    elplot.xaxis.axis_line_width=2
    elplot.yaxis.axis_line_width=2
    elplot.line(xxx_norm, yyy_norm, color='blue')
    from bokeh.models import ColumnDataSource, Ellipse
    #elplot.ellipse(x=xc, y=yc, width=a, height=b, angle=theta * 180 / np.pi + 90 , fill_color="yellow", fill_alpha = 0.4)
    #source = ColumnDataSource(dict(x=xc, y=yc, w=a, h=b))
    #glyph = Ellipse(x="x", y="y", width="w", height="h", angle=theta, fill_color="#cab2d6")
    #elplot.add_glyph(glyph)
    ################ local slope
    local_slope = []
    for i in range(len(xxx)-1):
        slope = (yyy[i+1]-yyy[i])/(xxx[i+1]-xxx[i])
        local_slope.append(slope)
    mean_slope = np.mean(local_slope)
    #from scipy.signal import find_peaks
    #peaks, _ = find_peaks(local_slope, prominence=0.5)   
    np.insert(local_slope, 0, np.mean(local_slope))
    local_slope = np.array(local_slope)
    local_slope=np.insert(local_slope, 0,0)
    
    peak_indx = []
    peak_values = []
    slope_mean = np.mean(local_slope)
    for val in range(len(local_slope)):
        if local_slope[val] > np.percentile(local_slope,95) or local_slope[val] < np.percentile(local_slope,5):
            #print(local_slope[val])
            peak_values.append(local_slope[val])
            peak_indx.append(val)
    #sk = skew(peak_indx)
    def consecutive(data, stepsize=1):
        return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)
    

    consec = np.max([len(cc) for cc in consecutive(peak_indx)])
    
    sloplot = figure(x_axis_label = "Fault length (m)", y_axis_label= "Displacement (m)", width=500, height=250) 

    sloplot.scatter(xxx_norm[peak_indx],local_slope[peak_indx], color = 'red')    
    sloplot.line(xxx_norm, local_slope, color ='blue')    
    if len(peak_indx)/len(local_slope)*100 <=75: #and consec <4:#math.isclose(sk, 0,rel_tol=0.5):
    #if consec <3:#math.isclose(sk, 0,rel_tol=0.5):
        shape = 'plateau'
    else:
        shape = 'peaked'
    sloplot.title.text=f'Slope - Fault ID: {jj}, Type: {shape}'  
    sloplot.border_fill_color = None
    sloplot.output_backend = "svg"    
    sloplot.xaxis.axis_label_text_font_size = "15pt"
    sloplot.xaxis.major_label_text_font_size = "15pt"
    sloplot.xaxis.axis_label_text_font = "times"
    sloplot.xaxis.axis_label_text_color = "black"
    sloplot.yaxis.axis_label_text_font_size = "15pt"
    sloplot.yaxis.major_label_text_font_size = "15pt"
    sloplot.yaxis.axis_label_text_font = "times"
    sloplot.yaxis.axis_label_text_color = "black"
    sloplot.yaxis.major_tick_line_width = 2
    sloplot.yaxis.major_tick_out = 10
    sloplot.yaxis.minor_tick_line_width = 2
    sloplot.xaxis.major_tick_line_width = 2
    sloplot.xaxis.major_tick_out = 10
    sloplot.xaxis.minor_tick_line_width = 2
    sloplot.xaxis.axis_line_width=2
    sloplot.yaxis.axis_line_width=2    
    profiles.append([chplot, jj])
    if CV >=2.3:        
        fault_type = 'plateau'
    else:
        fault_type = 'peaked'
    cvs.append([jj, CV, fault_type])


In [ ]:
profiles.sort(key = lambda x: x[1])
sorted_profiles = [p[0] for p in profiles]
plot_chunks = []
for c in chunk_creator(sorted_profiles, 5):
    plot_chunks.append(c)
id_chunks=[]
for c in chunk_creator(transect_unified_sorted['Fault ID'].unique(), 5):
    id_chunks.append(c)

## Plotting and saving

In [ ]:
start_id = 0
stop_id = 0 
for i in range(len(plot_chunks)):
    grid_pair = []
    for c in chunk_creator(plot_chunks[i], 4):
        grid_pair.append(c)
    grid = gridplot(grid_pair, width=500, height=250)
    start_id = id_chunks[i][0]
    stop_id =id_chunks[i][-1]
    export_svg(grid, filename = f'./Atlas_Dec2022/processing/Atlas_Displacement_along_faults_{start_id}-{stop_id}_normalized.svg')
    export_png(grid, filename = f'./Atlas_Dec2022/processing/Atlas_Displacement_along_faults_{start_id}-{stop_id}_normalized.png')

In [ ]:
import pandas as pd
results = pd.DataFrame(cvs, columns=['Fault ID','CV', 'Type'])
results.to_excel(f'./Atlas_Dec2022/processing/Atlas_CV_classification.xlsx')

## Scaling LAW

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### Define Laws

In [ ]:
def powlaw(x, a, b) :
    return a * np.power(x, b)
def linlaw(x, a, b) :
    return a + x * b
from scipy.optimize import curve_fit

### Defining data

In [ ]:
x = faults_unified['Fault L(m)']
y = faults_unified['MaxDsp(m)']
w = faults_unified['MaxWidth(m)']
yerr = faults_unified['StdErDsp']
#x_log = np.log10(x)
#y_log = np.log10(y)

### numpy curve_fit

In [ ]:
def powlaw(x, c, p) :
    return c * np.power(x, p)
def linlaw(x, c, p) :
    return c + x * p
from scipy.optimize import curve_fit

In [ ]:
## Power-law fit log scale
popt_pow, pcov_pow = curve_fit(powlaw, x,y)
ydatafit_pow = powlaw(x, *popt_pow)
correlation_matrix_pow = np.corrcoef(y, ydatafit_pow)
correlation_xy_pow = correlation_matrix_pow[0,1]
r_squared_pow =correlation_xy_pow**2
#print(f"y = {round(popt[0],4)}x^{round(popt[1],4)}\nR^2: {round(r_squared,4)}")
print("y=$%.6fx^{%.6f}$ \n$R^{2}$: %.6f"%(popt_pow[0],popt_pow[1],r_squared_pow))


In [ ]:
## Linear fit log scale
popt_lin, pcov_lin = curve_fit(linlaw, x,y)
ydatafit_lin = linlaw(x, *popt_lin)
correlation_matrix_lin = np.corrcoef(y, ydatafit_lin)
correlation_xy_lin = correlation_matrix_lin[0,1]
r_squared_lin = correlation_xy_lin**2
#print(f"y = {round(popt[0],4)}x^{round(popt[1],4)}\nR^2: {round(r_squared,4)}")
print("y=$%.6fx^{%.6f}$ \n$R^{2}$: %.6f"%(popt_lin[0],popt_lin[1],r_squared_lin))

In [ ]:
# plot mapped fit onto X
fig, ax = plt.subplots()
ax.scatter(x, y, s=50)#, cmap='rainbow')#, c = np.random.rand(50))


line1 = ax.plot(sorted(x), sorted(ydatafit_lin), color='red', label='Linear-scaling')
line2 = ax.plot(sorted(x), sorted(ydatafit_pow),color='green',label='Power-scaling')
# xerr = np.std(x)
# yerr = np.std(y)
#ax.errorbar(x, y, xerr=None, yerr=yerr, fmt='',linestyle='')
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel('Length (m)')
ax.set_ylabel('Max Displacement (m)')
ax.set_title("Linear y=$%.6fx^{%.6f}$ - $R^{2}$: %.6f\nPower y=$%.6fx^{%.6f}$ - $R^{2}$: %.6f"%(popt_lin[0],popt_lin[1],r_squared_lin,
                                                                                                popt_pow[0],popt_pow[1],r_squared_pow))
ax.legend()
fname = './Atlas_Dec2022/processing/Atlas_MaxDisp_Lengt_curvefit.png'
plt.savefig(fname, format='png')

## DMax/Length ratios frequencies

In [ ]:
DmaxL_ratios = [y[i]/x[i] for i in range(len(x))]
dmax_histograms = []
for i in [10,20,30,40,50,60]:    
    frequencies, edges = np.histogram(DmaxL_ratios, i)
    plot = figure(x_axis_label = 'DMax/L', y_axis_label= 'Frequency', width=500, height=250)
    plot.quad(top=frequencies, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
    plot.background_fill_color = None
    plot.border_fill_color = None
    plot.output_backend = "svg"
    plot.title.text=title=f"{i} bins"
    dmax_histograms.append(plot)

In [ ]:
grid_pair = []
plot_chunks = []
for c in chunk_creator(dmax_histograms, 1):
    plot_chunks.append(c)
id_chunks=[]
for c in chunk_creator(plot_chunks[0], 2):
    grid_pair.append(c)
grid = gridplot(grid_pair, width=500, height=250)
export_svg(grid, filename = f'./Atlas_Dec2022/processing/Atlas_DmaxL_freq.svg')
export_png(grid, filename = f'./Atlas_Dec2022/processing/Atlas_DmaxL_freq.png')

In [ ]:
i = 20
frequencies, edges = np.histogram(DmaxL_ratios, i)
plot = figure(x_axis_label = 'DMax/L', y_axis_label= 'Frequency', width=500, height=250)
plot.quad(top=frequencies, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
plot.background_fill_color = None
plot.border_fill_color = None
plot.output_backend = "svg"
plot.title.text=title=f"{i} bins"
plot.xaxis.axis_label_text_font_size = "15pt"
plot.xaxis.major_label_text_font_size = "15pt"
plot.xaxis.axis_label_text_font = "times"
plot.xaxis.axis_label_text_color = "black"
plot.yaxis.axis_label_text_font_size = "15pt"
plot.yaxis.major_label_text_font_size = "15pt"
plot.yaxis.axis_label_text_font = "times"
plot.yaxis.axis_label_text_color = "black"
plot.yaxis.major_tick_line_width = 2
plot.yaxis.major_tick_out = 10
plot.yaxis.minor_tick_line_width = 2
plot.xaxis.major_tick_line_width = 2
plot.xaxis.major_tick_out = 10
plot.xaxis.minor_tick_line_width = 2
plot.xaxis.axis_line_width=2
plot.yaxis.axis_line_width=2
export_svg(plot, filename = f'./Atlas_Dec2022/processing/Atlas_DmaxL_freq.svg')
export_png(plot, filename = f'./Atlas_Dec2022/processing/Atlas_DmaxL_freq.png')

## Width/Length ratios

In [ ]:
WmaxL_ratios = [w[i]/x[i] for i in range(len(x))]
wmax_histograms = []
for i in [10,20,30,40,50,60]:    
    frequencies, edges = np.histogram(WmaxL_ratios, i)
    plot = figure(x_axis_label = 'WMax/L', y_axis_label= 'Frequency', width=500, height=250)
    plot.quad(top=frequencies, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
    plot.background_fill_color = None
    plot.border_fill_color = None
    plot.output_backend = "svg"
    plot.title.text=title=f"{i} bins"
    wmax_histograms.append(plot)

In [ ]:
grid_pair = []
plot_chunks = []
for c in chunk_creator(wmax_histograms, 1):
    plot_chunks.append(c)
id_chunks=[]
for c in chunk_creator(plot_chunks[0], 2):
    grid_pair.append(c)
grid = gridplot(grid_pair, width=500, height=250)
export_svg(grid, filename = f'./Atlas_Dec2022/processing/Atlas_WmaxL_freq.svg')
export_png(grid, filename = f'./Atlas_Dec2022/processing/Atlas_WmaxL_freq.png')

In [ ]:
i = 20
frequencies, edges = np.histogram(WmaxL_ratios, i)
plot = figure(x_axis_label = 'WMax/L', y_axis_label= 'Frequency', width=500, height=250)
plot.quad(top=frequencies, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
plot.background_fill_color = None
plot.border_fill_color = None
plot.output_backend = "svg"
plot.title.text=title=f"{i} bins"
plot.xaxis.axis_label_text_font_size = "15pt"
plot.xaxis.major_label_text_font_size = "15pt"
plot.xaxis.axis_label_text_font = "times"
plot.xaxis.axis_label_text_color = "black"
plot.yaxis.axis_label_text_font_size = "15pt"
plot.yaxis.major_label_text_font_size = "15pt"
plot.yaxis.axis_label_text_font = "times"
plot.yaxis.axis_label_text_color = "black"
plot.yaxis.major_tick_line_width = 2
plot.yaxis.major_tick_out = 10
plot.yaxis.minor_tick_line_width = 2
plot.xaxis.major_tick_line_width = 2
plot.xaxis.major_tick_out = 10
plot.xaxis.minor_tick_line_width = 2
plot.xaxis.axis_line_width=2
plot.yaxis.axis_line_width=2
export_svg(plot, filename = f'./Atlas_Dec2022/processing/Atlas_WmaxL_freq.svg')
export_png(plot, filename = f'./Atlas_Dec2022/processing/Atlas_WmaxL_freq.png')